In [106]:
import random

# Generate 5 random 5-bit S-boxes (permutations of 0-31)
sboxes = [random.sample(range(32), 32) for _ in range(100)]

In [107]:
import random

# --- utility: dot product mod 2 of two 4-bit ints ---
def dot2(a: int, b: int) -> int:
    # parity of ones in (a & b)
    return bin(a & b).count("1") & 1

# # --- test the Fourier coefficient divisibility property on a multiset M ---
# def has_integral(M: list[int], beta: int, v: int) -> bool:
#     total = sum(1 if dot2(beta, x)==0 else -1 for x in M)
#     return (total % (2**v)) == 0

# --- test the divisibility property on a multiset M ---
def has_integral(M: list[int], beta: int, v: int) -> bool:
    total = sum(dot2(beta, x) for x in M)
    return (total % (2**v)) == 0



# --- build inverse S-box ---
def inverse_S(S: list[int]) -> list[int]:
    invS = [0]*32
    for x in range(32):
        invS[S[x]] = x
    return invS

In [132]:
def find_linear_structures(Sbox):
    """
    Given Sbox: a list of 16 ints in [0..15],
    returns a list of (beta, delta) pairs where
      Sbox[x^beta] ^ Sbox[x] == delta  for all x.
    """
    invS = inverse_S(Sbox)
    n = 5
    N = 1 << n
    lin = {}
    for beta in range(1, N):
        found = []
        # collect all output‐differences for this beta
        for w in range(1,N):
            deltas = { dot2(beta, invS[x ^ w] ^ invS[x]) for x in range(N) }
            # if there's exactly one, it's constant => a linear structure
            if len(deltas) == 1: 
            #and list(deltas)[0]==0:
                found.append(w)
        if found:
            lin[beta] = found
    return lin

# build your master list: one sub‐list per S‐box
all_structures = []
sboxes_with_linear_structures = [[2, 0, 9, 7, 18, 31, 24, 16, 12, 1, 6, 28, 25, 11, 13, 21, 17, 10, 27, 14, 15, 3, 19, 4, 5, 20, 23, 29, 22, 26, 8, 30], [13, 26, 21, 8, 2, 4, 16, 29, 10, 28, 23, 14, 20, 27, 11, 15, 9, 7, 25, 19, 0, 3, 5, 31, 12, 6, 24, 22, 17, 1, 18, 30], [2, 28, 25, 6, 1, 10, 24, 27, 17, 8, 4, 18, 16, 19, 7, 21, 9, 22, 31, 26, 23, 20, 29, 13, 12, 30, 3, 5, 14, 11, 15, 0], [21, 25, 22, 16, 11, 14, 10, 13, 8, 19, 6, 2, 15, 26, 4, 29, 0, 23, 17, 5, 1, 3, 24, 7, 30, 20, 31, 27, 28, 18, 9, 12], [15, 30, 3, 20, 6, 11, 22, 8, 4, 10, 1, 21, 29, 0, 25, 23, 7, 2, 14, 26, 28, 13, 18, 5, 12, 17, 19, 31, 24, 16, 27, 9], [28, 13, 4, 26, 19, 15, 17, 14, 21, 18, 9, 0, 3, 20, 16, 23, 7, 10, 6, 8, 11, 2, 5, 1, 30, 29, 12, 24, 31, 27, 25, 22], [1, 9, 15, 21, 29, 8, 30, 25, 5, 4, 17, 16, 28, 2, 31, 10, 22, 24, 18, 19, 26, 11, 13, 0, 23, 27, 6, 3, 14, 20, 12, 7], [8, 28, 24, 10, 2, 26, 4, 29, 30, 19, 21, 17, 12, 18, 14, 0, 3, 6, 9, 27, 25, 20, 5, 31, 7, 16, 15, 1, 23, 22, 11, 13], [3, 13, 12, 18, 11, 17, 7, 20, 10, 16, 15, 1, 22, 27, 26, 30, 23, 25, 2, 29, 24, 21, 4, 9, 31, 14, 8, 0, 5, 6, 28, 19], [13, 2, 0, 18, 27, 23, 10, 15, 14, 19, 21, 20, 22, 31, 30, 24, 5, 8, 3, 29, 7, 11, 26, 6, 12, 28, 25, 4, 16, 1, 9, 17], [30, 18, 21, 3, 8, 25, 19, 28, 31, 17, 4, 14, 16, 0, 22, 7, 2, 15, 20, 23, 10, 24, 13, 5, 1, 26, 27, 6, 11, 12, 9, 29], [3, 0, 1, 13, 20, 23, 6, 5, 28, 8, 22, 4, 24, 15, 14, 12, 27, 19, 17, 30, 26, 21, 31, 10, 7, 25, 29, 18, 11, 2, 9, 16], [4, 25, 1, 19, 20, 0, 30, 6, 9, 31, 17, 24, 21, 14, 29, 7, 28, 26, 27, 11, 15, 10, 3, 12, 13, 18, 8, 16, 5, 23, 22, 2], [4, 11, 29, 14, 6, 16, 2, 19, 27, 26, 24, 12, 20, 23, 10, 3, 30, 5, 31, 13, 22, 21, 7, 1, 28, 25, 9, 15, 8, 18, 0, 17], [21, 10, 31, 13, 22, 6, 0, 30, 20, 3, 24, 15, 7, 1, 12, 4, 19, 27, 28, 16, 18, 25, 9, 29, 2, 23, 14, 17, 5, 11, 26, 8], [0, 7, 8, 19, 27, 14, 21, 2, 22, 3, 15, 23, 11, 9, 16, 17, 13, 29, 18, 24, 31, 25, 30, 10, 5, 12, 1, 6, 4, 26, 28, 20], [2, 29, 12, 24, 9, 1, 28, 22, 0, 26, 13, 30, 25, 5, 10, 17, 4, 3, 16, 27, 11, 8, 15, 6, 18, 21, 31, 7, 23, 14, 20, 19], [26, 27, 9, 5, 28, 14, 22, 8, 25, 12, 11, 6, 3, 29, 31, 24, 13, 4, 17, 20, 15, 10, 1, 30, 18, 2, 23, 16, 0, 21, 19, 7], [4, 12, 19, 11, 15, 5, 0, 20, 31, 27, 8, 3, 7, 23, 9, 21, 24, 29, 1, 30, 6, 26, 28, 2, 14, 17, 10, 13, 25, 22, 18, 16], [29, 31, 26, 1, 9, 6, 0, 10, 19, 22, 4, 24, 23, 27, 13, 3, 11, 28, 16, 2, 8, 25, 17, 30, 18, 12, 21, 14, 20, 7, 15, 5]]
# print(len(sboxes_with_linear_structures))

# while len(sboxes_with_linear_structures)<10:
#     sbox = random.sample(range(32), 32)
for sbox in sboxes_with_linear_structures:
    linear_structures =  find_linear_structures(sbox)
    if linear_structures:
#         sboxes_with_linear_structures.append(sbox)
        all_structures.append(linear_structures)
    else:
        all_structures.append({})
        
# for i in range (10):
#     sbox = random.sample(range(32), 32)
# #     sboxes_with_linear_structures.append(sbox)
#     all_structures.append(find_linear_structures(sbox))
# example: print how many structures each S‐box has
for idx, struct in enumerate(all_structures):
    print(f"S-box #{idx:3d} has linear structures: {struct}")

S-box #  0 has linear structures: {22: [4, 9, 13]}
S-box #  1 has linear structures: {20: [31]}
S-box #  2 has linear structures: {12: [9]}
S-box #  3 has linear structures: {10: [2]}
S-box #  4 has linear structures: {20: [17]}
S-box #  5 has linear structures: {27: [12]}
S-box #  6 has linear structures: {18: [1]}
S-box #  7 has linear structures: {6: [3]}
S-box #  8 has linear structures: {28: [22]}
S-box #  9 has linear structures: {10: [28]}
S-box # 10 has linear structures: {}
S-box # 11 has linear structures: {}
S-box # 12 has linear structures: {}
S-box # 13 has linear structures: {}
S-box # 14 has linear structures: {}
S-box # 15 has linear structures: {9: [16]}
S-box # 16 has linear structures: {}
S-box # 17 has linear structures: {}
S-box # 18 has linear structures: {}
S-box # 19 has linear structures: {4: [21]}


In [133]:

v_list    = [5]
num_trials = 1000
results = {}  # will map (beta,v,idx) -> list of valid keys

All_vector = list(range(32))
for v in v_list:
    for idx, Sbox in enumerate(sboxes_with_linear_structures[:]):
        inv = inverse_S(Sbox)
        for beta in all_structures[idx].keys():
        #print(f"Sbox number {idx}")
            valid_keys = list(range(1, 32))
            
            
            trials = 0
            while trials < num_trials:
                M = [random.randrange(32) for _ in range(64)]
                while not has_integral(M, beta, v): 
                    M = [random.randrange(32) for _ in range(64)]
                # extend until the integral property holds
#                 while not has_integral(M, beta, v):
#                     M.extend(All_vector)
#                 # if we've already used this M, skip it
                trials += 1 
             
            
#            code for generating M randomly, and subset of M is all_vector
#             for _ in range(num_trials):
#                 # generate a valid M
#                 M = list(range(16))
#                 while not has_integral(M, beta, v) or M == All_vector:
#                     M.append(random.randrange(16))




                # test each key on a copy of valid_keys
                for k in valid_keys.copy():
                    M_prime = [inv[Sbox[x] ^ k] for x in M]
                    if not has_integral(M_prime, beta, v):
                        valid_keys.remove(k)
            
            results[(beta, v, idx)] = valid_keys





In [134]:
# Step 2: regroup by S-box index
results_by_sbox = {}
for (beta, v, sidx), keys in results.items():
    results_by_sbox.setdefault(sidx, {})[(beta, v)] = keys

# Step 3: print grouped by S-box
for sidx in sorted(results_by_sbox):
    print(f"\nS-box #{sidx:3d}:")
    bv_map = results_by_sbox[sidx]
    if not bv_map:
        print("  (no (β,v) pairs found)")
        continue
    for (beta, v), keys in sorted(bv_map.items()):
        print(f"  β = {beta:05b}, v = {v}  →  preserving keys: {keys}")


S-box #  0:
  β = 10110, v = 5  →  preserving keys: [4, 9, 13]

S-box #  1:
  β = 10100, v = 5  →  preserving keys: [31]

S-box #  2:
  β = 01100, v = 5  →  preserving keys: [9]

S-box #  3:
  β = 01010, v = 5  →  preserving keys: [2]

S-box #  4:
  β = 10100, v = 5  →  preserving keys: [17]

S-box #  5:
  β = 11011, v = 5  →  preserving keys: [12]

S-box #  6:
  β = 10010, v = 5  →  preserving keys: [1]

S-box #  7:
  β = 00110, v = 5  →  preserving keys: [3]

S-box #  8:
  β = 11100, v = 5  →  preserving keys: [22]

S-box #  9:
  β = 01010, v = 5  →  preserving keys: [28]

S-box # 15:
  β = 01001, v = 5  →  preserving keys: [16]

S-box # 19:
  β = 00100, v = 5  →  preserving keys: [21]


In [135]:
from itertools import chain, repeat

d = {0: 3, 1: 3, 2: 3, 3: 3, 4: 1, 5: 1, 6: 1, 7: 1, 8: 3, 9: 3, 10: 3, 11: 3, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 3, 21: 3, 22: 3, 23: 3, 24: 1, 25: 1, 26: 1, 27: 1, 28: 3, 29: 3, 30: 3, 31: 3}

flattened = list(chain.from_iterable(repeat(k, v) for k, v in d.items()))
print(flattened)
print(len(flattened))

v = 5
for i in range(32):
    print(i,has_integral(flattened, i, v))

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 5, 6, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 25, 26, 27, 28, 28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31]
64
0 True
1 True
2 True
3 True
4 True
5 True
6 True
7 True
8 True
9 True
10 True
11 True
12 True
13 True
14 True
15 True
16 True
17 True
18 True
19 True
20 False
21 True
22 True
23 True
24 True
25 True
26 True
27 True
28 True
29 True
30 True
31 True


In [136]:
M = flattened
n = 5
m = n+1
v_list    = [n-1]
# num_trials = 100000
results = {}  # will map (beta,v,idx) -> list of valid keys

# M = []
# for i in range(32):
#     if i<16:
#         M.append(i)
#         M.append(i)
#         M.append(i)
#     else:
#         M.append(i)
        
beta = 20
All_vector = list(range(32))
for v in v_list:
    for idx, Sbox in enumerate(sboxes_with_linear_structures):
        inv = inverse_S(Sbox)
        #print(f"Sbox number {idx}")
        valid_keys = list(range(1, 32))
            
            

                # test each key on a copy of valid_keys
        for k in valid_keys.copy():
            M_prime = [inv[Sbox[x] ^ k] for x in M]
            if not has_integral(M_prime, beta, v):
                valid_keys.remove(k)
        results[(beta, v, idx)] = valid_keys




In [137]:
# Step 2: regroup by S-box index
results_by_sbox = {}
for (beta, v, sidx), keys in results.items():
    results_by_sbox.setdefault(sidx, {})[(beta, v)] = keys

# Step 3: print grouped by S-box
for sidx in sorted(results_by_sbox):
    print(f"\nS-box #{sidx:3d}:")
    bv_map = results_by_sbox[sidx]
    if not bv_map:
        print("  (no (β,v) pairs found)")
        continue
    for (beta, v), keys in sorted(bv_map.items()):
        print(f"  β = {beta:05b}, v = {v}  →  preserving keys: {keys}")


S-box #  0:
  β = 10100, v = 4  →  preserving keys: [1, 2, 3, 7, 9, 10, 11, 12, 13, 14, 18, 21, 28, 30, 31]

S-box #  1:
  β = 10100, v = 4  →  preserving keys: [2, 3, 4, 5, 6, 8, 9, 14, 15, 16, 17, 22, 23, 25, 26, 27, 28, 29, 31]

S-box #  2:
  β = 10100, v = 4  →  preserving keys: [3, 4, 8, 9, 14, 15, 17, 19, 20, 22, 23]

S-box #  3:
  β = 10100, v = 4  →  preserving keys: []

S-box #  4:
  β = 10100, v = 4  →  preserving keys: [2, 4, 5, 6, 7, 8, 9, 10, 11, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27]

S-box #  5:
  β = 10100, v = 4  →  preserving keys: [3, 5, 6, 12, 13, 15, 16, 20, 22, 24, 25, 29]

S-box #  6:
  β = 10100, v = 4  →  preserving keys: [2, 8, 9, 12, 15, 16, 26, 27, 29, 30, 31]

S-box #  7:
  β = 10100, v = 4  →  preserving keys: [3, 5, 9, 10, 11, 14, 18, 19, 25, 27, 29, 30]

S-box #  8:
  β = 10100, v = 4  →  preserving keys: [2, 3, 6, 7, 8, 10, 11, 14, 15, 19, 20, 21, 26, 28, 29]

S-box #  9:
  β = 10100, v = 4  →  preserving keys: [2, 3, 4, 5, 9, 11, 15, 17, 21, 23, 24, 

In [144]:

idx = 4
Sbox = sboxes_with_linear_structures[idx]
inv = inverse_S(Sbox)
beta = 20 
v = n-1

keys_left = results[(beta, v, idx)] 

for delta in keys_left:
    D = 0
    for x in range(32):
        D += (-1)**(dot2(beta, inv[Sbox[x]]) ^ dot2(beta, inv[Sbox[x] ^ delta]))
    print("Key:", delta, "; Sum:", D)

Key: 2 ; Sum: 0
Key: 4 ; Sum: 0
Key: 5 ; Sum: 0
Key: 6 ; Sum: 0
Key: 7 ; Sum: 0
Key: 8 ; Sum: 0
Key: 9 ; Sum: 0
Key: 10 ; Sum: 0
Key: 11 ; Sum: 0
Key: 17 ; Sum: 32
Key: 19 ; Sum: 0
Key: 20 ; Sum: 0
Key: 21 ; Sum: 0
Key: 22 ; Sum: 0
Key: 23 ; Sum: 0
Key: 24 ; Sum: 0
Key: 25 ; Sum: 0
Key: 26 ; Sum: 0
Key: 27 ; Sum: 0


In [ ]:
len(sboxes_with_linear_structures)